### steps
1. Create an sample SQL database
2. performing SQL window on database

In [1]:
import sqlite3
from sqlite3 import Error

In [3]:
#Create table train schedule
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def create_station_table(conn, stations):
    """
    Create a new task
    :param conn:
    :param employees:
    :return:
    """
    sql_station_table = '''
                        CREATE TABLE IF NOT EXISTS stations_schedule
                        (
                        [Train_id] INTERGER,
                        [Station] TEXT,
                        [Time] INTERGER
                        )
                        ;'''
                        #INTERGER PRIMARY KEY need to be unique
    sql = '''
            INSERT INTO stations_schedule(Train_id, Station, Time)
            VALUES(?,?,?)
            ;'''
    cur = conn.cursor()
    cur.execute(sql_station_table)
    cur.executemany(sql, stations)
    conn.commit()
    #return cur.lastrowid
    
def main():
    database = r"./sql_win_lead.db"
    
    conn = create_connection(database)
    with conn:
        task = [(110, 'San Francisco', "10:00:00"),
                (110, 'Redwood City', "10:54:00"),
                (110, 'Palo Alto', "11:02:00"),
                (110, 'San Jose', "12:35:00"),
                (120, 'San Francisco', "11:00:00"),
                (120, 'Redwood City', "Non Stop"),
                (120, 'Palo Alto', "12:49:00"),
                (120, 'San Jose', "13:30:00"),
                ]
        #create station table with query task
        create_station_table(conn, task)

if __name__=='__main__':
    main()

In [4]:
#from sqlite3 import connect
import pandas as pd

In [5]:
#connect to database
conn = sqlite3.connect('sql_win_lead.db')
pd.read_sql("""SELECT *
                FROM stations_schedule
                """, conn)

,Train_id,Station,Time
0,110,San Francisco,10:00:00
1,110,Redwood City,10:54:00
2,110,Palo Alto,11:02:00
3,110,San Jose,12:35:00
4,120,San Francisco,11:00:00
5,120,Redwood City,Non Stop
6,120,Palo Alto,12:49:00
7,120,San Jose,13:30:00
8,110,San Francisco,10:00:00
9,110,Redwood City,10:54:00


In [6]:
#add a new column called "time to next station". To obtain this value, we substact 
#the station times for pairs of contigous stations. Calulating this value without 
#using a sql window function will be very complicated.
#Use LEAD window function instead.

In [7]:
#LEAD
#use TIME() and JULIANDAY() to format time to correct times in hh:mm:ss
#station to station distance is not exceeding 24 hours
lead_time_query = '''
SELECT
            train_id,
            station,
            time as station_time,
            LEAD(time) OVER (PARTITION BY train_id ORDER BY time)
                        AS lead_time,
            time((JULIANDAY(LEAD(time) OVER (PARTITION BY train_id ORDER BY time)) - JULIANDAY(time)) * 86400, 'unixepoch')  
                                        AS minutes_to_next_station
FROM stations_schedule

;'''
pd.read_sql(lead_time_query, conn)

,Train_id,Station,station_time,lead_time,minutes_to_next_station
0,110,San Francisco,10:00:00,10:00:00,00:00:00
1,110,San Francisco,10:00:00,10:54:00,00:54:00
2,110,Redwood City,10:54:00,10:54:00,00:00:00
3,110,Redwood City,10:54:00,11:02:00,00:08:00
4,110,Palo Alto,11:02:00,11:02:00,00:00:00
5,110,Palo Alto,11:02:00,12:35:00,01:33:00
6,110,San Jose,12:35:00,12:35:00,00:00:00
7,110,San Jose,12:35:00,None,None
8,120,San Francisco,11:00:00,11:00:00,00:00:00
9,120,San Francisco,11:00:00,12:49:00,01:49:00


In [8]:
#add column shows elapsed time travel
lead_elapsed_time_query = '''
SELECT
            train_id,
            station,
            time as station_time,
            TIME((JULIANDAY(time) - min(JULIANDAY(time)) OVER (PARTITION BY train_id ORDER BY time)) * 86400, 'unixepoch')
                                AS elapsed_travel_time,
            LEAD(time) OVER (PARTITION BY train_id ORDER BY time)
                        AS lead_time,
            TIME((JULIANDAY(LEAD(time) OVER (PARTITION BY train_id ORDER BY time)) - JULIANDAY(time)) * 86400, 'unixepoch')  
                                        AS minutes_to_next_station
FROM stations_schedule

;'''
pd.read_sql(lead_elapsed_time_query, conn)

,Train_id,Station,station_time,elapsed_travel_time,lead_time,minutes_to_next_station
0,110,San Francisco,10:00:00,00:00:00,10:00:00,00:00:00
1,110,San Francisco,10:00:00,00:00:00,10:54:00,00:54:00
2,110,Redwood City,10:54:00,00:54:00,10:54:00,00:00:00
3,110,Redwood City,10:54:00,00:54:00,11:02:00,00:08:00
4,110,Palo Alto,11:02:00,01:02:00,11:02:00,00:00:00
5,110,Palo Alto,11:02:00,01:02:00,12:35:00,01:33:00
6,110,San Jose,12:35:00,02:35:00,12:35:00,00:00:00
7,110,San Jose,12:35:00,02:35:00,None,None
8,120,San Francisco,11:00:00,00:00:00,11:00:00,00:00:00
9,120,San Francisco,11:00:00,00:00:00,12:49:00,01:49:00
